#                                     Toronto's Neighborhoods

First we import the libraries


In [171]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Then we download the Wikipedia page and then find the table

In [172]:
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(url).text
soup = BeautifulSoup(source,'lxml')
soup.prettify()
table=soup.find('table')
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_

then, we perse the table into a list

In [185]:
table_not_persed1=table.findAll('td')

data=[]
for i in range(len(table_not_persed1)):
    table_not_persed=table.findAll('td')[i].text
    data.append(table_not_persed)

len(data)

861

then we convert it into an array then split it into array for each row

In [186]:
data=np.array(data)
data=np.split(data,287)#Splitting each row into an array
data

[array(['M1A', 'Not assigned', 'Not assigned\n'], dtype='<U50'),
 array(['M2A', 'Not assigned', 'Not assigned\n'], dtype='<U50'),
 array(['M3A', 'North York', 'Parkwoods\n'], dtype='<U50'),
 array(['M4A', 'North York', 'Victoria Village\n'], dtype='<U50'),
 array(['M5A', 'Downtown Toronto', 'Harbourfront\n'], dtype='<U50'),
 array(['M6A', 'North York', 'Lawrence Heights\n'], dtype='<U50'),
 array(['M6A', 'North York', 'Lawrence Manor\n'], dtype='<U50'),
 array(['M7A', 'Downtown Toronto', "Queen's Park\n"], dtype='<U50'),
 array(['M8A', 'Not assigned', 'Not assigned\n'], dtype='<U50'),
 array(['M9A', "Queen's Park", 'Not assigned\n'], dtype='<U50'),
 array(['M1B', 'Scarborough', 'Rouge\n'], dtype='<U50'),
 array(['M1B', 'Scarborough', 'Malvern\n'], dtype='<U50'),
 array(['M2B', 'Not assigned', 'Not assigned\n'], dtype='<U50'),
 array(['M3B', 'North York', 'Don Mills North\n'], dtype='<U50'),
 array(['M4B', 'East York', 'Woodbine Gardens\n'], dtype='<U50'),
 array(['M4B', 'East York', 'P

We now convert it into a dataframe

In [187]:
data= pd.DataFrame(data)

In [188]:
data.rename({0:'Postal Code',1:'Borough',2:'Neighborhood'}, inplace=True, axis=1)

In [189]:
data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


we now replace the not assigned into NaN (not important)

In [190]:
data.replace({'Not assigned': 'NaN'},inplace=True)

In [191]:
data.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,Not assigned\n
1,M2A,NaN,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


we can now drop the NaN values

In [192]:
data.drop(data.loc[data['Borough']=='NaN'].index,axis=0,inplace=True)

In [193]:
data.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M6A,North York,Lawrence Heights\n
6,M6A,North York,Lawrence Manor\n


We reset the index

In [194]:
data.reset_index(inplace=True)
data.drop('index',axis=1,inplace=True)
data.shape

(210, 3)

We remove the '\n' in the end of the Neiborhood column and check for any not assigned neighborhoods

In [195]:
data['Neighborhood']=data['Neighborhood'].str.split("\n")
data['Neighborhood']
for i in range(210):
    
    data['Neighborhood'][i]=data['Neighborhood'][i][0]

In [196]:
data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [199]:
i=0
for i,j in zip(list(data['Neighborhood']=='Not assigned'),range(210)):
    if i == True:
        print(data.iloc[j,:])
        
data.iloc[6,2]= data.iloc[6,1]
data.head(7)

Postal Code              M9A
Borough         Queen's Park
Neighborhood    Not assigned
Name: 6, dtype: object


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park


We try to geocode the postal codes 

In [110]:
import geocoder # import geocoder
latitude=[]
longitude=[]
# initialize your variable to None
lat_lng_coords = None
for postal_code in data.iloc[:,0]:
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0]) 
    longitude.append(lat_lng_coords[1]) 

Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/geocode/json?address=M3A%2C+Toronto%2C+Ontario&bounds=&components=&region=&language= (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')")))
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/geocode/json?address=M3A%2C+Toronto%2C+Ontario&bounds=&components=&region=&language= (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')")))
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/geocode/json?address=M3A%2C+Toronto%2C+Ontario&bounds=&components=&region=&language= (Caused by SSLError(S

KeyboardInterrupt: 

In [200]:
#latitude=np.array(latitude)
#longitude=np.array(longitude)
#data['lat']=latitude.T
#data['lng']=longitude.T
data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


Since the geocoder did not work we download list from the provided link

In [201]:
#since the geocoder doesn't work
import wget
wget.download('https://cocl.us/Geospatial_data')

'Geospatial_Coordinates (1).csv'

In [202]:
toronto_data = pd.read_csv('Geospatial_Coordinates.csv')

In [203]:
toronto_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We then merge the coordinates with the neighborhood data 

In [204]:
merged_inner = pd.merge(left=data,right=toronto_data,left_on='Postal Code',right_on='Postal Code')

In [205]:
merged_inner.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [129]:
#merged_inner.drop('Postal Code',axis=1,inplace=True)

We finally cluster the data using K Means setting K = 5

In [206]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(merged_inner[['Latitude','Longitude']])
kmeans.labels_[0:10]

array([3, 3, 1, 2, 2, 1, 4, 3, 3, 2])

In [207]:
merged_inner['Cluster']= kmeans.labels_

In [208]:
merged_inner.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,3
1,M4A,North York,Victoria Village,43.725882,-79.315572,3
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1
3,M6A,North York,Lawrence Heights,43.718518,-79.464763,2
4,M6A,North York,Lawrence Manor,43.718518,-79.464763,2


We now draw the map with the clusters

In [209]:
# create map
latitude=43.6532
longitude=-79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_inner['Latitude'], merged_inner['Longitude'], merged_inner['Neighborhood'],
                                  merged_inner['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters